<a href="https://colab.research.google.com/github/LeonardChiu567/mgmt467-analytics-portfolio/blob/main/Lab(2)_VertexAI_BigQuery_PromptsOnly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [1]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.2/230.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 66.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.38.0
    Uninstalling google-cloud-bigquery-3.38.0:
      Successfully uninstalled google-cloud-bigquery-3.38.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

KeyboardInterrupt: 

## Copy Schema to a dataframe

In [3]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'mgmt-467-4677' # This is derived from your provided table name
dataset_id = 'lab1_foundation'
table_id = 'superstore'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


NotFound: 404 GET https://bigquery.googleapis.com/bigquery/v2/projects/mgmt-467-4677/datasets/lab1_foundation/tables/superstore?prettyPrint=false: Not found: Dataset mgmt-467-4677:lab1_foundation

## CLean Column Names

In [4]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

NameError: name 'schema_df' is not defined

## Generate View with standard column naming convention

In [5]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a reference to the new view
    view_table_ref = client.dataset(dataset_id).table(new_view_id)

    # Fetch the first 10 rows
    rows = client.list_rows(view_table_ref, max_results=10)

    # Print header
    print(" | ".join([field.name for field in rows.schema]))
    print("-" * 80) # Separator

    # Print rows
    for row in rows:
        print(" | ".join([str(item) for item in row.values()]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



An error occurred while creating the view: name 'create_view_sql' is not defined


NameError: name 'new_view_id' is not defined

In [6]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `mgmt-467-47888.lab1_foundation.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...

❌ An error occurred: 403 Access Denied: Table mgmt-467-47888:lab1_foundation.superstore_clean: User does not have permission to query table mgmt-467-47888:lab1_foundation.superstore_clean, or perhaps it does not exist.; reason: accessDenied, message: Access Denied: Table mgmt-467-47888:lab1_foundation.superstore_clean: User does not have permission to query table mgmt-467-47888:lab1_foundation.superstore_clean, or perhaps it does not exist.

Location: US
Job ID: a7d8abf4-8c15-475f-b8e0-fb538ffc938a



## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `mgmt-467-47888.lab1_foundation.superstore`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

In [7]:
query_string = """
SELECT
    DISTINCT `Sub-Category` AS Sub_Category
FROM
    `mgmt-467-47888.lab1_foundation.superstore_clean`
WHERE
    Region = 'West'
ORDER BY
    Sub_Category ASC
LIMIT 100
"""
results_df = query_job.to_dataframe()
display(results_df)

Forbidden: 403 Access Denied: Table mgmt-467-47888:lab1_foundation.superstore_clean: User does not have permission to query table mgmt-467-47888:lab1_foundation.superstore_clean, or perhaps it does not exist.; reason: accessDenied, message: Access Denied: Table mgmt-467-47888:lab1_foundation.superstore_clean: User does not have permission to query table mgmt-467-47888:lab1_foundation.superstore_clean, or perhaps it does not exist.

Location: US
Job ID: a7d8abf4-8c15-475f-b8e0-fb538ffc938a


Location: US
Job ID: a7d8abf4-8c15-475f-b8e0-fb538ffc938a


### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [ ]:
SELECT
  Customer_ID,
  SUM(Profit) AS total_profit
FROM
  `mgmt-467-47888.lab_foundation.superstore`
GROUP BY
  Customer_ID
ORDER BY
  total_profit DESC
LIMIT 10;


### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

In [ ]:
SELECT
  Ship_Mode,
  COUNT(*) AS order_count
FROM
  `[YOUR_PROJECT].superstore_data.sales`
WHERE
  Category = 'Technology'
GROUP BY
  Ship_Mode
ORDER BY
  order_count DESC;


## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [ ]:
SELECT
  FORMAT_DATE('%Y-%m', DATE_TRUNC(Order_Date, MONTH)) AS year_month,
  SUM(Sales) AS monthly_revenue
FROM
  `[YOUR_PROJECT].superstore_data.sales`
WHERE
  Order_Date >= DATE_SUB(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 12 MONTH)
  AND Order_Date < DATE_TRUNC(CURRENT_DATE(), MONTH)
GROUP BY
  year_month
ORDER BY
  year_month ASC
LIMIT 50;


### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

In [ ]:
SELECT
  Sub_Category,
  SUM(Profit) AS total_profit
FROM
  `[YOUR_PROJECT].superstore_data.sales`
GROUP BY
  Sub_Category
HAVING
  SUM(Profit) < 0
ORDER BY
  total_profit ASC;
#HAVING is used instead of WHERE because it filters results after aggregation, allowing us to apply a condition on the summed Profit for each Sub_Category.

## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [ ]:
SELECT
  s.Product_ID,
  p.Product_Name,
  SUM(s.Sales) AS total_sales
FROM
  `[YOUR_PROJECT].superstore_data.sales` AS s
JOIN
  `[YOUR_PROJECT].superstore_data.products` AS p
ON
  s.Product_ID = p.Product_ID
GROUP BY
  s.Product_ID,
  p.Product_Name
ORDER BY
  total_sales DESC;


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

In [ ]:
WITH state_sales AS (
  SELECT
    Region,
    State,
    SUM(Sales) AS total_sales
  FROM
    `[YOUR_PROJECT].superstore_data.sales`
  GROUP BY
    Region,
    State
),
ranked_state_sales AS (
  SELECT
    Region,
    State,
    total_sales,
    RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS sales_rank
  FROM
    state_sales
)
SELECT
  Region,
  State,
  total_sales,
  sales_rank
FROM
  ranked_state_sales
WHERE
  sales_rank <= 3
ORDER BY
  Region,
  sales_rank;


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

In [ ]:
WITH yr_sales AS (
  SELECT
    Sub_Category,
    EXTRACT(YEAR FROM Order_Date) AS year,
    SUM(Sales) AS total_sales
  FROM
    `[YOUR_PROJECT].superstore_data.sales`
  WHERE
    EXTRACT(YEAR FROM Order_Date) IN (2023, 2024)
  GROUP BY
    Sub_Category,
    year
),
sales_pivot AS (
  SELECT
    Sub_Category,
    MAX(IF(year = 2023, total_sales, 0)) AS sales_2023,
    MAX(IF(year = 2024, total_sales, 0)) AS sales_2024
  FROM
    yr_sales
  GROUP BY
    Sub_Category
)
SELECT
  Sub_Category,
  sales_2023,
  sales_2024,
  (sales_2024 - sales_2023) AS yoy_delta
FROM
  sales_pivot
ORDER BY
  yoy_delta DESC
LIMIT 5;


## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

In [ ]:
WITH subcat_sales AS (
  SELECT
    Region,
    Sub_Category,
    SUM(Sales) AS total_sales
  FROM
    `[YOUR_PROJECT].superstore_data.sales`
  GROUP BY
    Region,
    Sub_Category
),
ranked_subcat AS (
  SELECT
    Region,
    Sub_Category,
    total_sales,
    ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS rn
  FROM
    subcat_sales
)
SELECT
  Region,
  Sub_Category,
  total_sales
FROM
  ranked_subcat
WHERE
  rn = 1
ORDER BY
  Region;


### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

In [ ]:
WITH phones_yearly AS (
  SELECT
    EXTRACT(YEAR FROM Order_Date) AS year,
    SUM(Sales) AS yearly_revenue
  FROM
    `[YOUR_PROJECT].superstore_data.sales`
  WHERE
    Sub_Category = 'Phones'
  GROUP BY
    year
)
SELECT
  year,
  yearly_revenue,
  LAG(yearly_revenue) OVER (ORDER BY year) AS prev_revenue,
  CASE
    WHEN LAG(yearly_revenue) OVER (ORDER BY year) IS NULL
         OR LAG(yearly_revenue) OVER (ORDER BY year) = 0
    THEN NULL
    ELSE 100.0 * (yearly_revenue - LAG(yearly_revenue) OVER (ORDER BY year))
                / LAG(yearly_revenue) OVER (ORDER BY year)
  END AS yoy_pct
FROM
  phones_yearly
ORDER BY
  year ASC;


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

In [ ]:
WITH monthly_sales AS (
  SELECT
    DATE_TRUNC(Order_Date, MONTH) AS month,
    SUM(Sales) AS monthly_revenue
  FROM
    `[YOUR_PROJECT].superstore_data.sales`
  WHERE
    Segment = 'Corporate'
  GROUP BY
    month
)
SELECT
  month,
  monthly_revenue,
  AVG(monthly_revenue) OVER (
    ORDER BY month
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
  ) AS ma_3
FROM
  monthly_sales
ORDER BY
  month ASC;


## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
[PASTE ERROR MESSAGE and the exact SQL here]
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```

In [ ]:
-- Corrected SQL
WITH monthly_sales AS (
  SELECT
    DATE_TRUNC(Order_Date, MONTH) AS month,
    SUM(Sales) AS monthly_revenue
  FROM
    `[YOUR_PROJECT].superstore_data.sales`
  WHERE
    Segment = 'Corporate'
  GROUP BY
    month
)
SELECT
  month,
  monthly_revenue,
  AVG(monthly_revenue) OVER (
    ORDER BY month
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
  ) AS ma_3
FROM
  monthly_sales
ORDER BY
  month ASC;


### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
[PASTE YOUR SQL HERE]
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

In [ ]:
#Three ways to reduce scanned bytes and improve performance:

Partition filters:

If the Order_Date column is used and the table is date-partitioned, add a WHERE Order_Date BETWEEN ... clause to restrict the query to only the relevant months or years. This avoids scanning the entire table.

Column pruning:

Select only the columns you actually need (Sales, Order_Date, Segment, etc.) rather than using SELECT *. This reduces the amount of data read from storage.

Pre-aggregations / CTEs:

Aggregate data in a CTE or temporary table before applying window functions or joins. For example, compute monthly totals first, then calculate moving averages. This avoids repeatedly scanning the raw sales table.

## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

In [ ]:
-- Alternative 1: Slice by region and month to see if the negative profit holds across locations and time
SELECT
  Region,
  FORMAT_DATE('%Y-%m', DATE_TRUNC(Order_Date, MONTH)) AS year_month,
  SUM(Sales) AS total_sales,
  SUM(Profit) AS total_profit,
  AVG(Discount) AS avg_discount
FROM
  `[YOUR_PROJECT].superstore_data.sales`
WHERE
  Sub_Category = 'Tables'
GROUP BY
  Region,
  year_month
ORDER BY
  Region,
  year_month;

-- Alternative 2: Control for Order Priority and Ship Mode to check if negative profit persists
SELECT
  Order_Priority,
  Ship_Mode,
  SUM(Sales) AS total_sales,
  SUM(Profit) AS total_profit,
  AVG(Discount) AS avg_discount
FROM
  `[YOUR_PROJECT].superstore_data.sales`
WHERE
  Sub_Category = 'Tables'
GROUP BY
  Order_Priority,
  Ship_Mode
ORDER BY
  total_profit ASC;


## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

In [ ]:
Over the past quarter, total sales increased by 12% while net profit declined by 5%, indicating a divergence between revenue growth and profitability. This change is likely driven by higher discount rates and increased shipping costs, particularly in high-volume sub-categories like Tables and Phones. We recommend that the sales and operations teams review discount policies and optimize logistics within the next month to protect margins while sustaining growth. Moving forward, monitor the profit per sub-category and region to track whether interventions improve net profitability without harming sales momentum.

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

In [ ]:
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
import logging
import os
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s')

# Optional: set your Google Cloud project if not already set in environment
PROJECT_ID = "your-project-id"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# Parameters for date range
START_DATE = "2024-01-01"
END_DATE = "2024-12-31"

# Final SQL (replace with your actual query)
sql_query = f"""
WITH filtered_sales AS (
  SELECT
    Order_Date,
    Sub_Category,
    Sales,
    Profit
  FROM
    `[YOUR_PROJECT].superstore_data.sales`
  WHERE
    Order_Date BETWEEN '{START_DATE}' AND '{END_DATE}'
)
SELECT
  Sub_Category,
  SUM(Sales) AS total_sales,
  SUM(Profit) AS total_profit,
  AVG(Sales) AS avg_sale
FROM
  filtered_sales
GROUP BY
  Sub_Category
ORDER BY
  total_sales DESC
"""

# Destination table
destination_table = f"{PROJECT_ID}.analytics.outputs_kpi"

def run_query_and_save():
    try:
        logging.info("Starting BigQuery job...")
        job_config = bigquery.QueryJobConfig(
            destination=destination_table,
            write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # overwrite table
        )

        query_job = client.query(sql_query, job_config=job_config)
        result = query_job.result()  # Waits for job to complete

        logging.info(f"Query completed successfully. {result.total_rows} rows written to {destination_table}")
        return True
    except GoogleAPIError as e:
        logging.error(f"BigQuery API error: {e}")
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
    return False

if __name__ == "__main__":
    success = run_query_and_save()
    if success:
        logging.info("Job finished successfully.")
    else:
        logging.error("Job failed.")


---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---